# Reading files

Run the following cell and watch the video. 

In [1]:
%%html
<h1>ho ho ho</h1> 

# The special problem of reading data

There are several problems involved with getting data into a program. 
* Several high-level data formats. 
* Each one has several subformats.  
* A multitude of tools for reading.
* Each tool has limits. 

## The basic formats: 
* **text**: human-readable and editable data. 
* **binary**: machine-readable data. 
     
## Text data
* Numbers in one of many printable formats. 
* Less space-efficient than binary formats. 
* Tends to be portable between machines.

## Binary data
* Much more efficiently stored. 
* Tends not to be portable between machines and software. 
* Several variants are incompatible with one another. 

We've already seen a really simple example of reading a file, 


In [2]:
# run this to see the file. 
%pycat data.txt

1,2,3
4,5,6
7,8,9

In [3]:
# read the file into a list
file = open('data.txt', 'r')
out = []
for line in file:
    numbers = line.strip().split(',')
    out.append(numbers)
file.close()
out

[['1', '2', '3'], ['4', '5', '6'], ['7', '8', '9']]

# Oops! This is not quite right. 
There is a profound difference between '1' and 1 (without the quotes). 
* `'1'` is a string. 
* `1` is an integer. 
* (`1.` is a floating point number.) 

# Internal and string representations of numbers
* In python, there are several formats for numbers. 
* `int`: an integer
* `float`: a floating point number. 
* `cfloat`: a complex number. 

# Internal representations of numbers share these attributes: 
* Binary representation. 
* Usually 4, 8, or 16 characters (bytes) long. 
* Expressed in base 2. 

# String representations of numbers are obviously different
* Variable length. 
* Digits in base 10. 

# Converting between strings and numbers
* `str(numb)`: gives the string version of a number. 
* `float(strng)` or `int(strng)`: gives the numeric version. 

# So, we can solve our little problem as follows: 

In [4]:
# read the file into a list
out2 = []
for row in out: 
    new_row = []
    for entry in row: 
        new_row.append(int(entry))
    out2.append(new_row)
out2

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# Alas, if it were only that easy...

The reality: 
* 90% of the typical data scientist's time is spent finding ways to read data.
* Data reading errors are very common. 
* Data formats are not so neat as the above. 

Thus, an enormous amount of time has been spent on libraries for reading data. 

One of the simplest is `numpy.loadtxt`. 

See https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html

For example, we can write:

In [5]:
import numpy as np
stuff = np.loadtxt('data.txt', delimiter=',')
stuff

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

# A few observations

* default conversion type is float. 
* default delimiter is any "whitespace". 

So, for this file: 

In [6]:
# run this to see the file
%pycat data2.txt

1       2  3
     4  5  6
  7  8    9

we might write, instead: 

In [7]:
stuff = np.loadtxt('data2.txt')  # default delimiter is whitespace 
stuff

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

# Non-numeric data
Obviously, we need to do something about non-numeric data in the file. 
If we try naively to parse a file with non-numeric data,

In [8]:
%pycat data3.txt

1,2,James
2,4,Alison
3,6,Shay

In [9]:
try:
    stuff = np.loadtxt('data3.txt', delimiter=',')
except Exception as e:
    print(e)

could not convert string to float: 'James'


... and we must get more clever. Let's tell `loadtxt` what kinds of objects it is looking for. 
* i4: an integer
* U20: a string of up to 20 characters (U means *unicode*)

In [10]:
stuff = np.loadtxt('data3.txt', delimiter=',', dtype={'names': ['apples', 'oranges', 'name'],
                                                      'formats': ['i4', 'i4', 'U20']})
stuff

array([(1, 2, 'James'), (2, 4, 'Alison'), (3, 6, 'Shay')],
      dtype=[('apples', '<i4'), ('oranges', '<i4'), ('name', '<U20')])

# Notice several things in this example
* `loadtxt` automatically chose to represent this as an array of tuples rather than an array of lists. 
* Indexing is the same, but math is different. 
* This is its way of saying *this is not a vector, matrix, or tensor.* 

# The curse of comma-separated values

Comma-separated values are a very common data representation, but have one **huge** problem. 
Microsoft excel often puts out files like this. 

In [11]:
%pycat data3.csv

10,20,Frank Furter
23,40,Curious George
19,20,"Trump, Donald"

Excel thinks it's being clever, because there is a comma in the data. 
* It surrounds the field containing commands with double quotes. 
* Our parser knows nothing about that. 
So, when we try to parse it naively, we get: 

In [12]:
try:
    stuff = np.loadtxt('data3.csv', delimiter=',', dtype={'names': ['apples', 'oranges', 'name'],
                                                          'formats': ['i4', 'i4', 'U20']})
except Exception as e:
    print(e)

Wrong number of columns at line 3


This parser won't handle this case. 

We can, however, outsmart Excel by outputting as tab-separated values: 

In [13]:
%pycat data4.txt

30	40	Mr. Trump
10	20	Don, the twitterer
5	15	Trump, Donald


In [14]:
stuff = np.loadtxt('data4.txt', delimiter='\t', dtype={'names': ['apples', 'oranges', 'name'],
                                                       'formats': ['i4', 'i4', 'U20']})
stuff

array([(30, 40, 'Mr. Trump'), (10, 20, 'Don, the twitterer'),
       ( 5, 15, 'Trump, Donald')],
      dtype=[('apples', '<i4'), ('oranges', '<i4'), ('name', '<U20')])

*This always works because Excel won't allow a tab character to be typed into a cell!*

# Fixed-width fields

Some files we need to load have fields that occupy a fixed number of characters. These kinds of files are found as output of scientific modeling programs in Fortran. For example, consider: 

In [15]:
%pycat data5.txt

#2345671234123456781234
    2.4  10    25.2  10
   12.8  20    20.2   5
   20.2  30    15.4  15

We parse this kind of file with a different method that supports a more general *delimiter* field consisting of the widths -- as integers -- of the fields we should find. See https://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html
For example: 

In [16]:
stuff = np.genfromtxt('data5.txt', delimiter=(7,4,8,4))
stuff

array([[ 2.4, 10. , 25.2, 10. ],
       [12.8, 20. , 20.2,  5. ],
       [20.2, 30. , 15.4, 15. ]])

The delimiter field says that there are numbers in the input that are 7, 4, 8, and 4 characters long. I included a comment line (starting with `#`) in the data to show you the column numbers. 

# Let's put these patterns into practice
First run this cell to login to the grading system

In [17]:
# Don't change this cell; just run it. 
from client.api.notebook import Notebook
ok = Notebook('Reading files.ok')
ok.auth(inline=True)

Assignment: Reading files
OK, version v1.14.15

Successfully logged in as alva.couch@gmail.com


1. **Write code to read data from the file `exer1.txt`.**
Put the data into the numpy array `exer1`

In [18]:
%pycat exer1.txt

25.0,25.5,10.4
10.0,15,25.5
9,2,7


In [19]:
# fill in details ... 
exer1 = np.loadtxt('exer1.txt', delimiter=',')
print(exer1)

[[25.  25.5 10.4]
 [10.  15.  25.5]
 [ 9.   2.   7. ]]


In [20]:
_ = ok.grade('q01')  # run this to check your answer

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



2. **Write code to read data from the file `exer2.txt`.** Put the result into `exer2`.

In [21]:
%pycat exer2.txt

25.0    25.5 10.4
10.0  15        25.5
    9 2 7


In [22]:
# fill in details ... 
exer2 = np.loadtxt('exer2.txt')
print(exer2)

[[25.  25.5 10.4]
 [10.  15.  25.5]
 [ 9.   2.   7. ]]


In [23]:
_ = ok.grade('q02')  # run this to check your work

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



2. **Write code to read data from the file `exer3.txt`.** Put the result into `exer3`.

In [24]:
%pycat exer3.txt

#23412341234
25.025.510.4
10.015  25.5
 9   2   7


In [25]:
# fill in details ... 
exer3 = np.genfromtxt('exer3.txt', delimiter=[4,4,4])
print(exer3)

[[25.  25.5 10.4]
 [10.  15.  25.5]
 [ 9.   2.   7. ]]


In [26]:
_ = ok.grade('q03')  # run this to check your answer

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



(A personal note: You might think this file format is really clueless. It contains, however, the exact same data as the other two files. This kind of column run-together is common in outputs of scientific computing, and *fixed-width parsing is the only way to unravel it!* These exercises are chosen from *my personal experience in frustrating data wrangling* to get data into python and other languages! -- Prof. Couch)

4. (Advanced) Write code to input the data in `exer4.txt`. Make the third column an integer rather than floating point. Hint: look at the `dtype` field in the documentation for `genfromtxt` and use it to define the field types. Consider allowing the system to figure it out via `dtype=None` 

In [27]:
%pycat exer4.txt

#23412341234
20.030.0   4
40.550.5  30
60.270.3  50


In [28]:
# fill in details ...
exer4 = np.genfromtxt('exer4.txt', delimiter=[4,4,4], dtype=None)
print(exer4)

[(20. , 30. ,  4) (40.5, 50.5, 30) (60.2, 70.3, 50)]


In [29]:
type(exer4[0][2])

numpy.int64

In [30]:
_ = ok.grade('q04')  # run this to check your answer

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



### Afterword: what is an nparray? 
In this example, the result will be an `nparray` of tuples rather than a regular `nparray` of numbers. Why? *A regular `nparray` requires elements of the same type!*. Compare the above with the output from exercise 3 above. All elements are float, so that's a regular `nparray`. When you violate that restriction, `numpy` automatically gives you a tuple rather than a list of values. 

5. (Advanced) As in problem 4, read data from `exer5.txt`, and additionally label the columns 'carbon', 'nitrogen', 'oxygen'. Put the result into exer5.

In [31]:
%pycat exer5.txt

#23412341234
20.030.0   4
40.550.5  30
60.270.3  50


In [32]:
# fill in details ...
exer5 = np.genfromtxt('exer5.txt', delimiter=[4, 4, 4], dtype=None,
                      names=['carbon', 'nitrogen', 'oxygen'])
exer5

array([(20. , 30. ,  4), (40.5, 50.5, 30), (60.2, 70.3, 50)],
      dtype=[('carbon', '<f8'), ('nitrogen', '<f8'), ('oxygen', '<i8')])

In [33]:
# use this to test your work 
for key in exer5.dtype.names:
    print('{} is {}'.format(key, exer5[key]))

carbon is [20.  40.5 60.2]
nitrogen is [30.  50.5 70.3]
oxygen is [ 4 30 50]


Note that these are real `ndarray` vectors, not `tuple`s, because they each contain a single type of elements. 

In [34]:
_ = ok.grade('q05')  # run this to check your answer

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



# When you are done with this notebook, 
* Save and checkpoint. 
* Change `ready` to `True` in the cell below. 
* Run the cell below. 

In [36]:
ready = False  # change to True when ready to submit
if not ready:
    raise Exception("change ready to True when ready to submit")
_ = ok.submit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'Reading files.ipynb'.
Submit... 100% complete
Submission successful for user: alva.couch@gmail.com
URL: https://okpy.org/cal/COMP205/su19/readingfiles/submissions/YWNzVO

